In [35]:
import requests
from bs4 import BeautifulSoup
#import pymysql
from selenium import webdriver as wb
from tqdm import tqdm_notebook as tq
from selenium.webdriver.common.keys import Keys
import time as time
import getpass
import urllib.request
import random

from time import sleep
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
driver = wb.Chrome()
driver.get("https://www.instagram.com/accounts/login/")
driver.maximize_window()

In [37]:
username = getpass.getpass("Input ID : ")# User ID
password = getpass.getpass("Input PWD : ")# User PWD, 특히 getpass를 통해서 비밀번호 정보를 숨길 수도 있다. 잘 배워두자 
hashTag = input("Input HashTag # : ")# Search #
 
element_id = driver.find_element_by_name("username")
element_id.send_keys(username)
element_password = driver.find_element_by_name("password")
element_password.send_keys(password)
 
sleep(1.5)


Input ID : ········
Input PWD : ········
Input HashTag # : #숙소추천


In [38]:
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()

In [39]:
driver.get('https://www.instagram.com/explore/tags/숙소추천/')

In [55]:
##데이터를 저장할 Dictionary
insta_dict = {'id':[],
              'date': [],
              'like': [],
              'text': [],
              'hashtag':[]}
 

#첫 번째 게시물 클릭 
first_post = driver.find_element_by_class_name('eLAPa')
first_post.click()

ElementClickInterceptedException: Message: element click intercepted: Element <div class="eLAPa">...</div> is not clickable at point (189, 445). Other element would receive the click: <div class="tR2pe" style="padding-bottom: 75%;"></div>
  (Session info: chrome=91.0.4472.114)


In [56]:
imgsrclist = []

In [57]:
seq = 0
start = time.time()
 
while True:
    try:
        if driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow'):
            if seq % 50 == 0:
                print('{}번째 수집 중'.format(seq), time.time() - start, sep = '\t')
            try:
                info_id = driver.find_element_by_css_selector('h2._6lAjh').text
                insta_dict['id'].append(info_id)
            except:
                info_id = driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
                insta_dict['id'].append(info_id)
 ## 시간정보 수집 
            time_raw = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
            time_info = pd.to_datetime(time_raw.get_attribute('datetime')).normalize()
            insta_dict['date'].append(time_info)
 ## like 정보 수집
            try:
                driver.find_element_by_css_selector('button.sqdOP.yWX7d._8A5w5')
                like = driver.find_element_by_css_selector('button.sqdOP.yWX7d._8A5w5').text
                insta_dict['like'].append(like)
            except:
                insta_dict['like'].append('영상')
 ##text 정보수집
            raw_info = driver.find_element_by_css_selector('div.C4VMK').text.split()
            text = []
            for i in range(len(raw_info)):
                ## 첫번째 text는 아이디니까 제외 
                if i == 0:
                    pass
                ## 두번째부터 시작 
                else:
                    if '#' in raw_info[i]:
                        pass
                    else:
                        text.append(raw_info[i])
            clean_text = ' '.join(text)
            insta_dict['text'].append(clean_text)
 ##hashtag 수집
            raw_tags = driver.find_elements_by_css_selector('a.xil3i')
            hash_tag = []
            for i in range(len(raw_tags)):
                if raw_tags[i].text == '':
                    pass
                else:
                    hash_tag.append(raw_tags[i].text)
            insta_dict['hashtag'].append(hash_tag)
            seq += 1
            if seq == 10:
                break
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(3)
            
            try:
                image_src_element = driver.find_element_by_css_selector("div._97aPb img")
                image_src = image_src_element.get_attribute("src")
                imgsrclist.append(image_src)
        
            except:
                continue
            
        else:
            break
            
    except:
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(5)
        

0번째 수집 중	0.006967306137084961


In [42]:
insta = pd.DataFrame(insta_dict)

In [58]:
imgsrclist

[None,
 None,
 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/201525992_236181608269362_811629409422089958_n.jpg?tp=1&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=106&_nc_ohc=rGNjJPSBm9IAX9qDciP&edm=AP_V10EBAAAA&ccb=7-4&oh=0bf9a2e24514eec13223626989c68619&oe=60DC074C&_nc_sid=4f375e',
 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/199832586_311461847321120_8792652658779325225_n.jpg?tp=1&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=108&_nc_ohc=HWfxer6cX3gAX9G6k5M&edm=AP_V10EBAAAA&ccb=7-4&oh=14a0851cbb2b18cd6c001d59b3c0964c&oe=60DB4BBD&_nc_sid=4f375e',
 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/199886618_1538472279829604_2157445929466946393_n.jpg?tp=1&_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=108&_nc_ohc=EVQb0Z-0kvwAX_KBZoq&edm=AP_V10EBAAAA&ccb=7-4&oh=226a988d7c764daeec559a6e7b73b077&oe=60DC9FF7&_nc_sid=4f375e']

In [43]:
insta

,id,date,like,text,hashtag
0,travelmenu1,2021-05-29 00:00:00+00:00,영상,[ 깊은 산속 통나무집에서 즐기는 힐링타임! ] 자연의 싱그러움과 별들이 가득찬 밤...,"[#삼척, #향초목원, #여행다녀왔습니다, #여행다녀왔습니다, #여행다녀왔습니다_국..."
1,datecourse123,2021-06-22 00:00:00+00:00,영상,[해외같다고 난리난 쏠비치 진도-낮🌞버전] 국내맞아? 해외같은 여행지 하나 추가요💕...,"[#쏠비치진도, #진도호텔, #쏠비치진도호텔앤리조트, #진도여행, #전남여행, #호..."
2,busansomang,2021-06-21 00:00:00+00:00,영상,"감성숙소 모음리스트✈️ 갬성, 힐링 모든 면에서 대박인 제쥬도 존예 숙소 다 모아와...","[#제주, #제주, #제주도, #숙소, #제주숙소, #숙소추천, #제주숙소추천, #..."
3,1amny,2021-06-06 00:00:00+00:00,영상,해발 400m에 위치한 프라이빗 통나무집🪵🌳 안녕 여러분~! 이 곳은 강원도 삼척 ...,[]
4,seorimyeonga,2021-06-01 00:00:00+00:00,영상,- 좋은 기운 가득💚🌳🍷 3주,"[#서림연가, #서림연가m1, #푸르름, #피크닉, #작은마당, #smallbutg..."
...,...,...,...,...,...
995,blue__the_ocean,2021-06-14 00:00:00+00:00,영상,바다뷰 바다뷰맛집 송정해수욕장 투썸플레이스 옆 해변 30초 거리 우사인볼트 3초컷 ...,"[#송정블루디오션, #부산송정, #부산송정해수욕장, #부산숙소, #송정숙소, #송정..."
996,blue__the_ocean,2021-06-14 00:00:00+00:00,영상,바다뷰 바다뷰맛집 송정해수욕장 투썸플레이스 옆 해변 30초 거리 우사인볼트 3초컷 ...,"[#송정블루디오션, #부산송정, #부산송정해수욕장, #부산숙소, #송정숙소, #송정..."
997,blue__the_ocean,2021-06-14 00:00:00+00:00,영상,바다뷰 바다뷰맛집 송정해수욕장 투썸플레이스 옆 해변 30초 거리 우사인볼트 3초컷 ...,"[#송정블루디오션, #부산송정, #부산송정해수욕장, #부산숙소, #송정숙소, #송정..."
998,blue__the_ocean,2021-06-14 00:00:00+00:00,영상,바다뷰 바다뷰맛집 송정해수욕장 투썸플레이스 옆 해변 30초 거리 우사인볼트 3초컷 ...,"[#송정블루디오션, #부산송정, #부산송정해수욕장, #부산숙소, #송정숙소, #송정..."


In [ ]:
insta.to_excel('insta.xlsx', index=False)

In [45]:
insta.to_csv('C:/Users/SM2136/insta', index = None)

PermissionError: [Errno 13] Permission denied: 'C:/Users/SM2136/insta'

In [51]:
insta.to_csv("./sample.csv", index=False, encoding="utf-8-sig")